In [2]:
import itertools as it
import pandas as pd
import requests as rq
from IPython.display import display

In [3]:
users = "../data/users.json"
host = "localhost"
partition_port = {"Partition": "5001"}
limit = 10
walks = 1000
walk_length = 100

In [4]:
# Import users
user_df = pd.read_json(users, orient="table")
display(user_df)

,user_id
0,4835761544
1,1148181892878852101
2,323451705
3,16635604
4,279324423
...,...
495,884234329177677824
496,1283409507473747968
497,2190617000
498,268098772


In [5]:
#Output files
output_baseline = f"../data/baseline_recommendations_for_{len(user_df)}_users.json"
output_single_partition= f"../data/recommendations_for_{len(user_df)}_users_single_partition.json"
outputs = [output_baseline, output_single_partition]

In [6]:
def get_recommendations(user_df, walks, walk_length, limit):
    rows = []
    hash_function = "Single"
    partition = partition_port.items()[0]
    port = partition_port.items()[1]

    for i, row in user_df.iterrows():
        user = row["user_id"]
        url = f"http://{host}:{port}/recommendation/salsa/{user}?walks={walks}&walk_length={walk_length}&limit={limit}"
        response = rq.get(url)

        response_body = response.json() if response.status_code == 200 else []
        recommendations = [res['id'] for res in response_body]
        hits = [res['hit'] for res in response_body]
        rows.append((hash_function, partition, user, recommendations, hits))
    
    return pd.DataFrame(rows, columns=["hash_function", "partition", "user", "recommendations", "hits"])

In [7]:
# Save results
for out in outputs:
    recommendations_df = get_recommendations(user_df, walks, walk_length, limit)
    recommendations_df["hash_function"] = "Single"
    recommendations_df["partition"] = "Partition"
    recommendations_df.to_json(out, index=False, orient="table")

TypeError: 'dict_items' object is not subscriptable

In [8]:
partition_port.items()

dict_items([('Partition', '5001')])